2025-10-23 22:48:41 Thursday

论文阅读笔记

## 📄 基本信息

* **标题**：DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning
* **作者**：DeepSeek-AI（[research@deepseek.com](mailto:research@deepseek.com)）
* **单位**：DeepSeek-AI
* **年份**：2025-01-22（arXiv:2501.12948v1）
* **论文链接**：arXiv:2501.12948v1（见文首信息）
* **模型参考**：DeepSeek-R1-Zero（纯 RL，无 SFT）与 DeepSeek-R1（冷启动 + 多阶段 RL）；并开源多款蒸馏稠密模型（1.5B–70B）


## 🎯 一句话总结

> 本文证明：**不用 SFT 和神经奖励模型**，仅靠**大规模强化学习（RL）**也能在 LLM 中**自发涌现强推理行为**；在此基础上引入**少量冷启动数据与多阶段训练**，得到的 DeepSeek-R1 在多项推理基准上**可比肩 OpenAI-o1-1217**，并且将推理模式**蒸馏**到更小的稠密模型上。


---
## 🧩 1. 研究背景与问题定义

### ⚠️ 现有问题

OpenAI o1 证明了：**只要在推理阶段拉长 Chain-of-Thought（CoT）就能显著增强模型的推理表现**。
但这种 *inference-time scaling* 方法存在明显缺陷——**高成本（推理时间长、算力消耗大）且缺乏可控性**。

因此，本文希望**把这种推理增强从推理阶段转移到训练阶段**：
通过 **post-training 强化（training-time reasoning optimization）** 让模型在**不额外增加推理开销**的前提下，
**自动形成更长、更高质量的思维链（CoT）**。



### 🎯 研究目标

* **主线目标**：在 post-training 阶段直接提升模型的推理能力，使推理增强成为模型固有特性。
* **探索性目标**：验证 **纯 RL （不依赖 SFT 或 Reward Model）** 是否足以诱导出推理能力，并观察其涌现规律。




## ⚙️ 2. 方法概述

#### 🌈 核心思想

本文的创新点不在算法层面，而在于**后训练（post-training）流程设计**。
DeepSeek-R1 系列提出了一条新的 **“强化推理式多阶段训练范式”**：
通过纯强化学习（RL）直接激励模型生成高质量推理链（CoT），
并在此过程中逐步提升语言可读性与通用能力。

整个方法分为两个阶段模型：


#### 🧩 (1) DeepSeek-R1-Zero —— 纯强化学习验证阶段

* **目的**：验证在无监督微调（SFT）与无神经奖励模型（reward model）的情况下，
  仅靠 RL 是否能诱导模型自动学会推理行为。

* **做法**：

  1. 以 DeepSeek-V3-Base 为起点；
  2. 使用 **GRPO（Group Relative Policy Optimization）** 作为优化算法（沿用自 V3 论文）；
  3. 仅采用**规则化奖励（rule-based reward）**：

     * **正确性奖励**：判断输出结果是否正确；
     * **格式奖励**：检查是否包含 `<think>...</think>` 推理片段；
  4. 不使用任何人类标注的 CoT 样例。

* **结果**：
  模型在纯 RL 下自动涌现出「长思维链」「自我验证」「反思」等推理特征，
  证明 **RL 本身能激发 reasoning 能力**；
  但同时输出文本存在**多语言混杂、格式不稳定、可读性差**的问题。






#### 🧩 (2) DeepSeek-R1 —— 多阶段强化推理训练
初始模型为 R1-Zero，此后每个阶段的初始模型均来自上一阶段的输出。

| 阶段                    | 数据来源（谁生成的）                                               | 数据主题 / 任务分布                                       | 训练模式（算法 + 奖励）                                  | 阶段目的                                              |
| --------------------- | -------------------------------------------------------- | ------------------------------------------------- | ---------------------------------------------- | ------------------------------------------------- |
| **1️⃣ 冷启动 SFT**       | 人工/规则 构造的 少量高质量 CoT 样例（约几千条）                             | 聚焦 **推理任务**：数学、逻辑、代码推理（如 AIME、MATH 类问题）           | **SFT**（交叉熵）                                   | 教模型输出 `<think>…</think>` 格式、建立基本推理模式和语言稳定性        |
| **2️⃣ 推理导向 RL**       | 上一步 SFT 模型 → 同样的 推理任务集合                                  | 仍是 **数学/逻辑/代码 类可验证问题**（带标准答案 → 可 rule-based 打分）   | **GRPO 强化学习**，奖励 = 正确性 + 格式 + 语言一致性            | 用 RL 信号诱导更长、更稳定的 CoT 链，提升纯推理性能                    |
| **3️⃣ 拒绝采样 + 补充 SFT** | 来自 2️⃣ 阶段 RL 模型 生成的 回答 (多样采样 + 质量筛选 + DeepSeek-V3 judge) | **混合任务域**：保留推理数据 + 新增 非推理任务（写作、对话、角色扮演、问答、代码解释 等） | **SFT**（交叉熵）                                   | 让模型在保持推理能力的同时，恢复 LLM 通用语言与交互能力                    |
| **4️⃣ 全场景 RL**        | 上一步 混合 SFT 模型 → 继续生成 多任务样本                               | **综合任务分布**：推理 + 通用对话 + 代码 + 知识 + 指令跟随             | **GRPO 强化学习**，同样 rule-based 奖励 (正确性 + 格式 + 语言) | 在全域任务上做 统一 RL 优化，得到 推理强 且 语言流畅 的最终 DeepSeek-R1 模型 |





#### 🔍 训练模板

R1/R1-Zero 都遵循统一的生成模板：

```
<|system|> You are a helpful and precise assistant. Please think step by step.
<|user|> [输入问题]
<|assistant|> <think> [模型在此生成推理步骤] </think> [最终答案]
```




### 🧭 小结一句话

> DeepSeek-R1 的核心创新是 **后训练流程的系统性革新**：
> 从无 SFT 的纯 RL 验证（R1-Zero），到冷启动-多阶段强化-蒸馏的推理优化框架，
> 证明了强化学习不仅能“调偏好”，还能“学推理”。
